# Nettoyage et préparation des données foncières

Dans cette section, nous procédons au nettoyage des données brutes issues du fichier foncier afin de :

- Corriger les types de données pour assurer la cohérence.
- Traiter les valeurs manquantes et aberrantes.
- Formater les colonnes clés pour faciliter l'analyse et l'intégration dans des outils comme MySQL ou Power BI.
- Ajouter des variables calculées utiles (ex : prix au m²).

Ce travail garantit des données propres, fiables et prêtes à être exploitées pour des analyses statistiques et modélisations.


In [ ]:
import pandas as pd

data = pd.read_csv("../data/ValeursFoncieres-2024.txt", 
                   sep="|", 
                   low_memory=False
                   )

In [72]:
data.shape[0]

2324463

In [73]:
data.head(5)

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2024,Vente,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,P,NaN,99.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,03/01/2024,Vente,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,S,NaN,115.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,08/01/2024,Vente,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,S,NaN,497.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2024,Vente,...,NaN,1.0,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2024,Vente,...,NaN,2.0,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


Supression des colonnes inutiles du DataFrame, dont les valeurs nulles excédent 50%

In [74]:
print("Pourcentage de valeur manquante")
for col in data.columns:
    percent_missing = data[col].isna().mean()*100
    print(f"{col} {round(percent_missing,2)}%")

Pourcentage de valeur manquante
Identifiant de document 100.0%
Reference document 100.0%
1 Articles CGI 100.0%
2 Articles CGI 100.0%
3 Articles CGI 100.0%
4 Articles CGI 100.0%
5 Articles CGI 100.0%
No disposition 0.0%
Date mutation 0.0%
Nature mutation 0.0%
Valeur fonciere 1.24%
No voie 42.19%
B/T/Q 95.89%
Type de voie 44.76%
Code voie 0.5%
Voie 0.5%
Code postal 0.51%
Commune 0.0%
Code departement 0.0%
Code commune 0.0%
Prefixe de section 94.0%
Section 0.0%
No plan 0.0%
No Volume 99.78%
1er lot 74.86%
Surface Carrez du 1er lot 93.04%
2eme lot 92.83%
Surface Carrez du 2eme lot 97.77%
3eme lot 98.64%
Surface Carrez du 3eme lot 99.73%
4eme lot 99.57%
Surface Carrez du 4eme lot 99.93%
5eme lot 99.83%
Surface Carrez du 5eme lot 99.98%
Nombre de lots 0.0%
Code type local 46.48%
Type local 46.48%
Identifiant local 100.0%
Surface reelle bati 46.55%
Nombre pieces principales 46.55%
Nature culture 25.6%
Nature culture speciale 95.17%
Surface terrain 25.6%


In [75]:
cols_to_drop = []
for col in data.columns:
    if data[col].isna().mean() > 0.5:
        cols_to_drop.append(col)

data.drop(cols_to_drop, axis=1, inplace=True)


In [76]:
cols_to_drop = ['No disposition', 'No voie', 'Type de voie', 'Code voie', 'Voie', 'Nombre de lots', 'Nature culture']
data.drop(cols_to_drop, axis=1, inplace=True)

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324463 entries, 0 to 2324462
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date mutation              object 
 1   Nature mutation            object 
 2   Valeur fonciere            object 
 3   Code postal                float64
 4   Commune                    object 
 5   Code departement           object 
 6   Code commune               float64
 7   Section                    object 
 8   No plan                    float64
 9   Code type local            float64
 10  Type local                 object 
 11  Surface reelle bati        float64
 12  Nombre pieces principales  float64
 13  Surface terrain            float64
dtypes: float64(7), object(7)
memory usage: 248.3+ MB


Conserver suelement les maisons et les appartements

In [78]:
data = data[(data["Type local"] == "Appartement") | (data["Type local"] == "Maison")]

Conversion en valeur numérique exploitable

In [79]:
data["Valeur fonciere"] = data["Valeur fonciere"].astype(str).str.replace(",",".",regex=False)
data["Surface reelle bati"] = data["Surface reelle bati"].astype(str).str.replace(",",".", regex=False)

data['Valeur fonciere'] = pd.to_numeric(data["Valeur fonciere"], errors="coerce")
data['Surface reelle bati']= pd.to_numeric(data["Surface reelle bati"], errors="coerce")

data.head(5)

,Date mutation,Nature mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Section,No plan,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
6,03/01/2024,Vente,329500.0,1170.0,GEX,01,173.0,AI,551.0,2.0,Appartement,89.0,4.0,NaN
78,03/01/2024,Vente,94500.0,1150.0,LAGNIEU,01,202.0,AC,198.0,2.0,Appartement,74.0,3.0,65.0
80,03/01/2024,Vente,94500.0,1150.0,LAGNIEU,01,202.0,AC,198.0,2.0,Appartement,32.0,2.0,65.0
81,03/01/2024,Vente,220000.0,1640.0,BOYEUX-SAINT-JEROME,01,56.0,C,2523.0,1.0,Maison,40.0,1.0,488.0
82,03/01/2024,Vente,220000.0,1640.0,BOYEUX-SAINT-JEROME,01,56.0,C,2524.0,1.0,Maison,80.0,2.0,858.0


Supression des doublons

In [80]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)


Supression des lignes dont 'Valeur fonciere' est nulle pour optimiser l'analyse

In [81]:
data['Valeur fonciere'].isna().sum()

1824

In [82]:
data = data.dropna(subset=['Valeur fonciere'])

Supression des instances où les valeurs sont nulles

In [83]:
col_nozero = ["Valeur fonciere",
              "Nombre pieces principales",
              "Surface terrain",
              "Surface reelle bati"
              ]
for col in col_nozero:
    data = data[data[col] != 0]

Nettoyage des valeurs aberrantes

In [84]:
data.describe()

,Valeur fonciere,Code postal,Code commune,No plan,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain
count,6.123680e+05,612305.000000,612368.000000,612368.000000,612368.000000,612332.000000,612332.000000,4.110370e+05
mean,3.004114e+05,35056.053225,219.589226,364.940515,1.373705,85.088692,3.571740,9.158664e+02
std,1.545148e+06,18834.331955,169.652569,493.317407,0.483787,46.259613,1.618221,4.123706e+03
min,4.000000e-01,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00
25%,1.090000e+05,19000.000000,88.000000,88.000000,1.000000,55.000000,2.000000,2.490000e+02
50%,1.760000e+05,34280.000000,187.000000,209.000000,1.000000,80.000000,4.000000,5.000000e+02
75%,2.770000e+05,50890.000000,309.000000,447.000000,2.000000,106.000000,5.000000,8.520000e+02
max,6.588889e+07,69930.000000,909.000000,8395.000000,2.000000,2033.000000,198.000000,1.039068e+06


In [85]:
col_to_clean = ['Valeur fonciere',
                'Surface reelle bati',
                'Nombre pieces principales',
                'Surface terrain']

def detect_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    born_inf = Q1 - 1.5 * IQR
    born_sup = Q3 + 1.5 * IQR
    return(df[feature] < born_inf) | (df[feature] > born_sup)

for col in col_to_clean:
    outliers_mask = detect_outliers(data,col)
    data = data[~outliers_mask]


In [86]:
data.describe()

,Valeur fonciere,Code postal,Code commune,No plan,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain
count,528826.000000,528773.000000,528826.000000,528826.000000,528826.000000,528794.000000,528794.000000,335181.000000
mean,182504.292724,35350.393562,220.864916,369.520941,1.403579,77.164822,3.385437,503.531945
std,107512.160467,18813.743445,169.685448,496.113216,0.490615,33.741408,1.394746,353.617068
min,1.000000,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,100000.000000,19430.000000,88.000000,91.000000,1.000000,52.000000,2.000000,215.000000
50%,162000.000000,34400.000000,188.000000,215.000000,1.000000,75.000000,3.000000,459.000000
75%,244000.000000,51130.000000,310.000000,452.000000,2.000000,99.000000,4.000000,702.000000
max,529000.000000,69930.000000,909.000000,8395.000000,2.000000,174.000000,7.000000,1653.000000


Conversion en date

In [87]:
data['Date mutation'] = pd.to_datetime(data['Date mutation'], errors='coerce', dayfirst=True)
data['Date mutation'] = data['Date mutation'].dt.strftime('%Y-%m-%d')
data['Date mutation'].dtype

dtype('O')

Calcul du prix au m²

In [88]:
data["prix_m2"] = (data["Valeur fonciere"] / data["Surface reelle bati"]).round(0)


Convertit les float en str  
Supprime le .0 à la   
Formatage des colonnes par des zéros  

In [89]:
col_to_str = ['Code commune', 'Code postal','No plan','Code type local']
for col in col_to_str:
    data[col] = data[col].astype(str)
    data[col] = data[col].str.replace(r'\.0$', '', regex=True)

data["Code commune"] = data["Code commune"].str.zfill(3)
data["Code departement"] = data["Code departement"].str.zfill(2)
data["Code postal"] = data["Code postal"].str.zfill(5)

In [90]:
data.head(5)



,Date mutation,Nature mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Section,No plan,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,prix_m2
6,2024-01-03,Vente,329500.0,01170,GEX,01,173,AI,551,2,Appartement,89.0,4.0,NaN,3702.0
78,2024-01-03,Vente,94500.0,01150,LAGNIEU,01,202,AC,198,2,Appartement,74.0,3.0,65.0,1277.0
80,2024-01-03,Vente,94500.0,01150,LAGNIEU,01,202,AC,198,2,Appartement,32.0,2.0,65.0,2953.0
81,2024-01-03,Vente,220000.0,01640,BOYEUX-SAINT-JEROME,01,056,C,2523,1,Maison,40.0,1.0,488.0,5500.0
82,2024-01-03,Vente,220000.0,01640,BOYEUX-SAINT-JEROME,01,056,C,2524,1,Maison,80.0,2.0,858.0,2750.0


In [91]:
data.to_csv("../clean_data/ValeursFoncieres_nettoyees.csv", index=False, encoding="utf-8")